In [444]:
# class Word defines a word in dictionary; 
class Word(object):
    def __init__(self, WordName=None, WordIndex=None, WordFrequency=None):
        self.WordName = WordName
        self.WordIndex = WordIndex
        self.WordFrequency = WordFrequency


# class Element defines a word in a bug corpus or in a code corpus; 
class Element(object):
    def __init__(self, ElementName=None, ElementIndex=None, ElementFrequency=None, ElementWeight=None):
        self.ElementName = ElementName
        self.ElementIndex = ElementIndex
        self.ElementFrequency = ElementFrequency
        self.ElementWeight = ElementWeight


# class Corpus defines a bug corpus or a code corpus; 
class Corpus(object):
    def __init__(self, FileName=None):
        self.FileName = FileName
        self.CorpusMembers = []

In [445]:
def generate_OneBugCorpus(FilePath, WordList):
    
    for i in range(len(WordList)):
        WordList[i].WordFrequency = 0
    
    with open(FilePath) as TextFile:
        ThisBugFile = TextFile.read().split(' ')    


    ThisBugCorpus = Corpus(FilePath) # this file delete word that is not in the word list; 

    for i in range(len(ThisBugFile)):
        flag = False
        for j in range(len(WordList)):
            if ThisBugFile[i] == WordList[j].WordName:
                flag = True
                WordList[j].WordFrequency = WordList[j].WordFrequency + 1
                break        
               

    for i in range(len(WordList)):
        if WordList[i].WordFrequency > 0:
            ThisBugCorpus.CorpusMembers.append(Element(WordList[i].WordName, 0, WordList[i].WordFrequency, 0))            
        
    return ThisBugCorpus

In [446]:
def generate_OneCodeCorpus(ThisCodeFile, WordList):
    
    for i in range(len(WordList)):
        WordList[i].WordFrequency = 0
    
    ThisCodeCorpus = Corpus(ThisCodeFile[0])
    
    for i in range(len(ThisCodeFile)):
        flag = False
        for j in range(len(WordList)):
            if ThisCodeFile[i] == WordList[j].WordName:
                flag = True                    
                WordList[j].WordFrequency = WordList[j].WordFrequency + 1
                break
    
    for i in range(len(WordList)):
        if WordList[i].WordFrequency > 0:
            ThisCodeCorpus.CorpusMembers.append(Element(WordList[i].WordName, 0, WordList[i].WordFrequency, 0))     
    
    return ThisCodeCorpus

In [454]:
def compute_BugCorpus_weight(CorpusNumber, MemberNumber):
    
    ThisWord = BugCorpuses[CorpusNumber].CorpusMembers[MemberNumber].ElementName
    # count ftd, the frequency of term t in document d;
    ftd = BugCorpuses[CorpusNumber].CorpusMembers[MemberNumber].ElementFrequency
#    print("ftd:", ftd)

    # #docs, total number of documents; 
    NumDocs = len(BugFiles) + len(CodeCorpuses);
#    print("#docs:", NumDocs)

    # count nt, the number of docuement that contains t;
    # count ntBug, the number of document that contains t in bug files; 
    ntBug = 0 
    for i in range(len(BugCorpuses)):    
        flag = False        
        for j in range(len(BugCorpuses[i].CorpusMembers)):
            if ThisWord == BugCorpuses[i].CorpusMembers[j].ElementName:
                flag = True
                break

        if flag == True:
            ntBug = ntBug + 1

#    print("ntBug:", ntBug)

    # count ntCode, the number of document that contains t in code files; 
    ntCode = 0
    for i in range(len(CodeCorpuses)):
        flag = False
        for j in range(len(CodeCorpuses[i].CorpusMembers)):
            if ThisWord == CodeCorpuses[i].CorpusMembers[j].ElementName:
                flag = True
                break

        if flag == True:
                ntCode = ntCode + 1

#    print("ntCode:", ntCode)

    nt = ntBug + ntCode
#    print("nt:", nt)

    weight = (math.log(ftd)+1) * math.log(NumDocs/nt)
#    print(weight)
    
    return weight

In [448]:
# this process intends to save all file names in an array; 

import os

path = 'C:\\Users\\take_\\OneDrive\\liming\\where\\BugLocator\\tmp\\BugCorpus\\'

BugFiles = []
# r=root, d=directories, f = bug files
for r, d, f in os.walk(path):
    for file in f:
        if '.txt' in file:
            BugFiles.append(os.path.join(r, file))

print(len(BugFiles))

98


In [449]:

## ----------------------------------- read word list from Wordlist.txt --------------------------------------------
with open("WordList.txt") as TextFile:
    WordlistContent = [line.split() for line in TextFile]

WordList = []
    
for i in range(len(WordlistContent)):    
    WordList.append(Word(WordlistContent[i][0], WordlistContent[i][1], 0))
    
# test whether WordList is correct or not;     
#print(WordList[0].WordName)
#print(WordList[1].WordName)
    
    
## ----------------------------------- read all bug files ------------------------------------------------    
#ThisBugCorpus = generate_OneBugCorpus(BugFiles[0], WordList)
#print(ThisBugCorpus.FileName)
#print(ThisBugCorpus.CorpusMembers)

BugCorpuses = []
for i in range(len(BugFiles)):
    ThisBugCorpus = generate_OneBugCorpus(BugFiles[i], WordList)
    BugCorpuses.append(ThisBugCorpus)

# test result of all bug corpuses;     
# print(BugCorpuses[0])
for i in range(1):
    print(BugCorpuses[i].FileName)
    for j in range(len(BugCorpuses[i].CorpusMembers)):
        print(BugCorpuses[i].CorpusMembers[j].ElementName, end = ',')
        print(BugCorpuses[i].CorpusMembers[j].ElementFrequency, end = ',')
    print()

## ----------------------------------- read all code files ------------------------------------------------
with open("CodeCorpus.txt") as TextFile:
    CodeList = TextFile.read().split()   

#print(len(CodeList))
CodeFiles = []
ThisCodeFile = []

for i in range(len(CodeList)):
    if CodeList[i][-5:] == ".java":        
        CodeFiles.append(ThisCodeFile)
        ThisCodeFile = []
        ThisCodeFile.append(CodeList[i])
    else:
        ThisCodeFile.append(CodeList[i])

CodeFiles.pop(0) # delete the first element;         

#print(len(CodeFiles))
#print(CodeFiles[0])

#ThisCodeCorpus = generate_OneCodeCorpus(CodeFiles[0], WordList)
#print(ThisCodeCorpus)

CodeCorpuses = []
for i in range(len(CodeFiles)):
    ThisCodeCorpus = generate_OneCodeCorpus(CodeFiles[i], WordList)
    CodeCorpuses.append(ThisCodeCorpus)

# test all code corpuses generated; 
print(CodeCorpuses[0].FileName)
#print(CodeCorpuses[0].CorpusMembers)
for i in range(len(CodeCorpuses[0].CorpusMembers)):
    print(CodeCorpuses[0].CorpusMembers[i].ElementName, end = ',')
    print(CodeCorpuses[0].CorpusMembers[i].ElementFrequency, end = ',')
print()

C:\Users\take_\OneDrive\liming\where\BugLocator\tmp\BugCorpus\100040.txt
averag,2,time,1,mirror,1,icon,2,includ,1,ilc,1,menu,1,perspect,3,test,1,numer,1,swt,2,ui,3,os,1,ms,6,number,1,window,1,increas,1,show,1,toolbar,1,run,1,perform,1,eclips,2,imagelist,2,make,1,elaps,6,process,6,singl,1,due,1,debug,4,build,1,
org.eclipse.swt.accessibility.ACC.java
client,1,access,1,scrollbar,1,radiobutton,1,listitem,1,acc,3,delet,1,deprec,1,multipl,1,tablecel,1,state,17,press,1,tabitem,1,insert,1,tablerowhead,1,focus,2,contain,1,combobox,1,dialog,1,tooltip,1,pushbutton,1,select,2,label,1,expand,1,separ,1,sizeabl,1,constant,1,hottrack,1,offscreen,1,treeitem,1,childid,3,multiselect,1,progressbar,1,menu,1,text,3,slider,1,readonli,1,tablecolumnhead,3,tree,1,menubar,1,checkbutton,1,check,1,defin,1,list,1,role,32,area,1,link,2,tablecolumn,1,sinc,1,xd,1,xc,1,xb,1,xa,1,us,2,invis,1,window,1,toolbar,1,object,1,tabfold,1,tabl,1,menuitem,1,busi,1,collaps,1,


In [464]:
import math

#compute_BugCorpus_weight(0, 3)

for i in range(len(BugCorpuses)):
    for j in range(len(BugCorpuses[i].CorpusMembers)):
        BugCorpuses[i].CorpusMembers[j].ElementWeight= compute_BugCorpus_weight(i, j)
    
    
# test result of element weight;
for i in range(2):
    print(BugCorpuses[i].FileName)
    for j in range(len(BugCorpuses[i].CorpusMembers)):
        print("[", end = '')
        print(BugCorpuses[i].CorpusMembers[j].ElementName, end = ',')
#        print(BugCorpuses[i].CorpusMembers[j].ElementFrequency, end = ',')
        print(BugCorpuses[i].CorpusMembers[j].ElementWeight, end = '')
        print("]", end = ',')
    print()

C:\Users\take_\OneDrive\liming\where\BugLocator\tmp\BugCorpus\100040.txt
[averag,8.051444535206512],[time,2.3944588432997893],[mirror,3.3690184832979204],[icon,5.075132358277732],[includ,2.963553375189756],[ilc,4.755312844417811],[menu,2.472930458741285],[perspect,9.979557971756522],[test,3.0325462466767075],[numer,3.879844107063911],[swt,4.369269789557716],[ui,6.601971790873969],[os,1.5525664014794938],[ms,14.701791934818253],[number,1.9827241221780296],[window,1.7300217686222754],[increas,3.656700555749701],[show,1.9827241221780296],[toolbar,3.9668554840535406],[run,2.4135070382704837],[perform,3.106654218830429],[eclips,3.960946933000165],[imagelist,8.051444535206512],[make,2.49354974594402],[elaps,13.898652784923842],[process,6.847425925475732],[singl,2.627081138568543],[due,4.167526179515692],[debug,11.347576225995212],[build,3.4203117776854706],
C:\Users\take_\OneDrive\liming\where\BugLocator\tmp\BugCorpus\100095.txt
[transfer,3.3690184832979204],[fulli,3.9668554840535406],[inter